# CS471 Final Project Data Processing
### Julia Razzino, Connor Brezenski, Michaela Kovalsky

We are designing a machine learning model to predict an individual's Big Five personality traits based on their extended Spotify listening history. First, we will import the required libraries.

In [4]:
# Import statements
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import numpy as np
import glob
import os


/Users/connorbrezenski/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Next, we create a function to read in the listening history data (json) and convert it to a csv.

In [2]:
def json_to_csv(json_filename, csv_filename=None):
    try:
        if csv_filename is None:
            csv_filename = json_filename.replace('.json', '.csv')
        
        with open(json_filename, encoding='utf-8') as inputfile:
            df = pd.read_json(inputfile)
        
        df.to_csv(csv_filename, encoding='utf-8', index=False)
        
        return f"CSV file '{csv_filename}' created successfully."
    
    except FileNotFoundError:
        return f"Error: The file '{json_filename}' does not exist."
    except ValueError as e:
        return f"Error: {e}. Ensure the JSON structure is valid for conversion."
    except Exception as e:
        return f"An unexpected error occurred: {e}"


Here, we call the function we just made and print the results.

In [5]:
result = json_to_csv('data/Kovalsky_Streaming_History_Audio_2014-2020.json')
print(result)

CSV file 'data/Kovalsky_Streaming_History_Audio_2014-2020.csv' created successfully.


Check for the Spotify Web API install and set up.

In [6]:
%pip install pandas spotipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Set up the spotify API.

In [ ]:
# Replace with your Spotify API credentials
CLIENT_ID = 'ff359b48e7074fe09b99477eb78af081'         # Replace with your Client ID
CLIENT_SECRET = 'b726f74b70a54c458a59839442f6a710' # Replace with your Client Secret

# Authenticate with Spotify
client_credentials_manager = SpotifyClientCredentials(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


Specify path listening history file path.

In [59]:
# Replace with your JSON file path
json_filename = '.\\data\\Brezenski_Streaming_History_Audio_2023-2024_8.json'

# Load the JSON data into a DataFrame
df = pd.read_json(json_filename)


Perpare data for Spotify API call.

In [60]:
# Ensure the column name matches your data structure
track_uri_column = 'spotify_track_uri'  # Adjust if your column name is different

# Drop rows without a track URI
df = df.dropna(subset=[track_uri_column])

# Extract unique track URIs
unique_track_uris = df[track_uri_column].unique().tolist()

print(f"Total unique tracks: {len(unique_track_uris)}")


Total unique tracks: 5407


Set up dictionaries.

In [61]:
# Dictionaries to hold fetched data
track_info_dict = {}
audio_features_dict = {}
artist_genres_dict = {}


Call track details and audio features.

In [62]:
# Spotify API allows fetching up to 50 tracks per request
BATCH_SIZE = 50

for i in range(0, len(unique_track_uris), BATCH_SIZE):
    batch_uris = unique_track_uris[i:i+BATCH_SIZE]
    
    # Fetch track details
    try:
        tracks = sp.tracks(batch_uris)['tracks']
        for track in tracks:
            if track:  # Check if track is not None
                track_id = track['uri']
                track_info_dict[track_id] = {
                    'track_name': track['name'],
                    'album_name': track['album']['name'],
                    'album_release_date': track['album']['release_date'],
                    'track_popularity': track['popularity'],
                    'duration_ms': track['duration_ms'],
                    'explicit': track['explicit'],
                    'artist_ids': [artist['id'] for artist in track['artists']],
                    'artist_names': [artist['name'] for artist in track['artists']],
                }
    except Exception as e:
        print(f"Error fetching track details for batch starting at index {i}: {e}")
    
    # Fetch audio features
    try:
        features = sp.audio_features(batch_uris)
        for feature in features:
            if feature:  # Check if feature is not None
                track_id = feature['uri']
                audio_features_dict[track_id] = feature
    except Exception as e:
        print(f"Error fetching audio features for batch starting at index {i}: {e}")
    
    time.sleep(0.1)  # Adjust sleep time as necessary


Fetch artist's generes.

In [63]:
# Collect all unique artist IDs
all_artist_ids = set()
for track_data in track_info_dict.values():
    all_artist_ids.update(track_data['artist_ids'])

print(f"Total unique artists: {len(all_artist_ids)}")

# Fetch artist genres in batches
artist_ids_list = list(all_artist_ids)
for i in range(0, len(artist_ids_list), BATCH_SIZE):
    batch_artist_ids = artist_ids_list[i:i+BATCH_SIZE]
    try:
        artists = sp.artists(batch_artist_ids)['artists']
        for artist in artists:
            if artist:  # Check if artist is not None
                artist_id = artist['id']
                artist_genres_dict[artist_id] = artist['genres']
    except Exception as e:
        print(f"Error fetching artist genres for batch starting at index {i}: {e}")
    
    time.sleep(0.1)  # Adjust sleep time as necessary


Total unique artists: 3542


Add dictionaries to data frame.

In [64]:
# Convert track info dict to DataFrame
track_info_df = pd.DataFrame.from_dict(track_info_dict, orient='index')

# Convert audio features dict to DataFrame
audio_features_df = pd.DataFrame.from_dict(audio_features_dict, orient='index')

# Reset index to have 'spotify_track_uri' as a column
track_info_df = track_info_df.reset_index().rename(columns={'index': 'spotify_track_uri'})
audio_features_df = audio_features_df.reset_index().rename(columns={'index': 'spotify_track_uri'})


Print results.

In [65]:
print(track_info_df)

                         spotify_track_uri  \
0     spotify:track:7aqfrAY2p9BUSiupwk3svU   
1     spotify:track:7LTCmtMpf3JPfBE8gAl64z   
2     spotify:track:0VJXWgw0GWsprapvlpbuyu   
3     spotify:track:4M68xjcc42oxyphhzpOWXS   
4     spotify:track:5VE6OSfJkMHyMpHaJzuUqn   
...                                    ...   
5402  spotify:track:688ucfDoelAC8sY7wgHCV4   
5403  spotify:track:2wAJTrFhCnQyNSD3oUgTZO   
5404  spotify:track:3mTpegrOwRn0oJjv4TSbEE   
5405  spotify:track:6gbiTbclnHlmSIPfmF2zEc   
5406  spotify:track:5A8xI7PN4WDe9e61xEdt94   

                                track_name                      album_name  \
0     First Person Shooter (feat. J. Cole)                For All The Dogs   
1           Turn Yo Clic Up (feat. Future)  Turn Yo Clic Up (feat. Future)   
2                                     Okay                            Okay   
3                             Great Gatsby                       Nostalgia   
4                            Gimme Da Lite               

Merge results into one data frame.

In [ ]:
# Merge track info and audio features on 'spotify_track_uri'
track_data_df = pd.merge(track_info_df, audio_features_df, on='spotify_track_uri', how='left')

Review columns for correctness.

In [67]:
print(track_data_df.columns)

Index(['spotify_track_uri', 'track_name', 'album_name', 'album_release_date',
       'track_popularity', 'duration_ms_x', 'explicit', 'artist_ids',
       'artist_names', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms_y', 'time_signature'],
      dtype='object')


Get genres for artists.

In [68]:
# Function to get genres for a list of artist IDs
def get_genres_for_artists(artist_ids):
    genres = set()
    for artist_id in artist_ids:
        artist_genres = artist_genres_dict.get(artist_id, [])
        genres.update(artist_genres)
    return list(genres)

# Apply the function to each row
track_data_df['artist_genres'] = track_data_df['artist_ids'].apply(get_genres_for_artists)


Convert to strings.

In [69]:
# Flatten 'artist_ids' and 'artist_names' to strings
track_data_df['artist_ids'] = track_data_df['artist_ids'].apply(lambda x: ', '.join(x))
track_data_df['artist_names'] = track_data_df['artist_names'].apply(lambda x: ', '.join(x))
track_data_df['artist_genres'] = track_data_df['artist_genres'].apply(lambda x: ', '.join(x))

# Optionally, drop columns you don't need
# track_data_df = track_data_df.drop(columns=['type', 'id', 'track_href', 'analysis_url', 'time_signature'])


Merge data into data frame.

In [70]:
# Merge the enriched track data back into the original DataFrame
df_enriched = pd.merge(df, track_data_df, on='spotify_track_uri', how='left')


Print results.

In [71]:
print(df_enriched)

                         ts    username platform  ms_played conn_country  \
0      2023-12-22T15:58:39Z  kingzenski      ios     109100           US   
1      2023-12-22T15:58:40Z  kingzenski      ios       1010           US   
2      2023-12-22T15:58:56Z  kingzenski      ios      16620           US   
3      2023-12-22T15:59:00Z  kingzenski      ios       4050           US   
4      2023-12-22T15:59:06Z  kingzenski      ios       5700           US   
...                     ...         ...      ...        ...          ...   
16909  2024-05-31T22:45:15Z  kingzenski      ios      34943           US   
16910  2024-05-31T22:45:19Z  kingzenski      ios       3370           US   
16911  2024-05-31T22:45:22Z  kingzenski      ios       3541           US   
16912  2024-05-31T22:45:25Z  kingzenski      ios       2560           US   
16913  2024-05-31T22:45:29Z  kingzenski      ios       3754           US   

                           ip_addr_decrypted user_agent_decrypted  \
0                 

Create output file.

In [72]:
# Replace with your desired output file path
output_csv_filename = 'data/your_streaming_history_enriched.csv'

# Save to CSV
df_enriched.to_csv(output_csv_filename, index=False)

print(f"Enriched data saved to {output_csv_filename}")


Enriched data saved to data/your_streaming_history_enriched.csv


In [14]:
import pandas as pd
import numpy as np
import glob
import os

# Define the data directory
data_dir = 'data/processed/'  # Replace with your actual directory path

# Get a list of all CSV files
csv_files = glob.glob(os.path.join(data_dir, '*.csv'))

# Initialize a list to hold data from each user
user_data_list = []

for file in csv_files:
    # Extract the username from the filename
    filename = os.path.basename(file)
    name_part = os.path.splitext(filename)[0]
    name_parts = name_part.split('_')
    username = '_'.join(name_parts[:-1])  # Joins all parts except the last one (assuming date is last)
    
    # Read the CSV file
    df_user = pd.read_csv(file, quotechar='"', quoting=1, encoding='utf-8')
    
    # Add the username column
    df_user['username'] = username
    
    # Debugging: Print out the columns in each user file
    print(f"Columns in {file}:", df_user.columns.tolist())
    
    # Append to the list
    user_data_list.append(df_user)

# Combine all user data into a single DataFrame
df = pd.concat(user_data_list, ignore_index=True)

# Check columns
print("Columns in df after combining user data:", df.columns.tolist())

# Proceed with the rest of your data processing...
assert 'username' in df.columns, "'username' column is missing in df"


Columns in data/processed/Kovalsky_2014-2020.csv: ['ts', 'username', 'platform', 'ms_played', 'conn_country', 'ip_addr_decrypted', 'user_agent_decrypted', 'master_metadata_track_name', 'master_metadata_album_artist_name', 'master_metadata_album_album_name', 'spotify_track_uri', 'episode_name', 'episode_show_name', 'spotify_episode_uri', 'reason_start', 'reason_end', 'shuffle', 'skipped', 'offline', 'offline_timestamp', 'incognito_mode', 'track_name', 'album_name', 'album_release_date', 'track_popularity', 'duration_ms_x', 'explicit', 'artist_ids', 'artist_names', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms_y', 'time_signature', 'artist_genres']
Columns in data/processed/Razzino_2023-2024.csv: ['ts', 'username', 'platform', 'ms_played', 'conn_country', 'ip_addr_decrypted', 'user_agent_decrypted', 'master_metadata_track_name', 'master

/var/folders/64/s8x7jmb54wq7ldv0c_5hb_wm0000gn/T/ipykernel_64053/1414443006.py:23: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_user = pd.read_csv(file, quotechar='"', quoting=1, encoding='utf-8')


In [15]:
# Check for missing values
print(df.isnull().sum())

# Decide on a strategy: drop or impute missing values
#df = df.dropna(subset=['essential_column1', 'essential_column2'])  # Replace with actual column names

df = df.drop_duplicates()

irrelevant_columns = ['ts', 'platform', 'conn_country', 'ip_addr_decrypted', 'user_agent_decrypted',
                      'spotify_track_uri', 'episode_name', 'episode_show_name', 'spotify_episode_uri',
                      'reason_start', 'reason_end', 'offline_timestamp', 'track_href', 'analysis_url',
                      'uri', 'id', 'track_name', 'album_name', 'album_release_date', 'duration_ms_y',
                      'type', 'master_metadata_track_name', 'master_metadata_album_artist_name',
                      'master_metadata_album_album_name']
df = df.drop(columns=irrelevant_columns, errors='ignore')  # Use errors='ignore' in case some columns are missing

ts                                       0
username                                 0
platform                                 0
ms_played                                0
conn_country                             0
ip_addr_decrypted                     1777
user_agent_decrypted                 10977
master_metadata_track_name               0
master_metadata_album_artist_name        0
master_metadata_album_album_name         0
spotify_track_uri                        0
episode_name                         58125
episode_show_name                    58125
spotify_episode_uri                  58125
reason_start                            22
reason_end                            1793
shuffle                                  0
skipped                               3796
offline                                  0
offline_timestamp                        0
incognito_mode                           0
track_name                             158
album_name                             158
album_relea

In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Assume 'df' is your DataFrame containing all user data

# Check if 'artist_genres' is in df.columns
if 'artist_genres' not in df.columns:
    print("Warning: 'artist_genres' column is missing from the DataFrame.")
    df['artist_genres'] = ''  # Create the column with empty strings

# Remove leading/trailing spaces and ensure the field is a string
df['artist_genres'] = df['artist_genres'].astype(str).str.strip()


# 1. Split 'artist_genres' into lists
df['artist_genres_list'] = df['artist_genres'].apply(
    lambda x: x.split(', ') if isinstance(x, str) and x else []
)

# Verify that all entries in 'artist_genres_list' are lists
assert df['artist_genres_list'].apply(lambda x: isinstance(x, list)).all(), "Not all entries are lists."

# 2. Aggregate genres per user
user_genres = df.groupby('username')['artist_genres_list'].apply(
    lambda lists: [genre for sublist in lists for genre in sublist]
)

# 3. Apply MultiLabelBinarizer to create dummy variables for genres
mlb = MultiLabelBinarizer()
genre_dummies = pd.DataFrame(
    mlb.fit_transform(user_genres),
    columns=mlb.classes_,
    index=user_genres.index
).reset_index()

# 4. Handle 'key' variable
df['key'] = df['key'].fillna(-1).astype(int)  # Fill missing 'key' values

# Create dummy variables for 'key'
key_dummies = pd.get_dummies(df['key'], prefix='key')

# Concatenate 'username' and key_dummies
df_keys = pd.concat([df[['username']], key_dummies], axis=1)

# Aggregate 'key' dummies per user (using mean to get the proportion of times each key was played)
user_keys = df_keys.groupby('username').mean().reset_index()

# 5. Aggregate other numerical features per user
agg_functions = {
    'ms_played': 'sum',
    'skipped': 'mean',
    'shuffle': 'mean',
    'explicit': 'mean',
    'danceability': 'mean',
    'energy': 'mean',
    'loudness': 'mean',
    'speechiness': 'mean',
    'acousticness': 'mean',
    'instrumentalness': 'mean',
    'liveness': 'mean',
    'valence': 'mean',
    'tempo': 'mean',
    # Add other features as needed
}

user_numerical_agg = df.groupby('username').agg(agg_functions).reset_index()

# 6. Prepare DataFrames for merging
def prepare_dataframe(df, name):
    if 'username' not in df.columns:
        df.reset_index(inplace=True)
    df['username'] = df['username'].astype(str)
    df.columns = df.columns.str.strip()
    return df


# Aggregate per user and reset index
user_agg = df.groupby('username').agg(agg_functions).reset_index()

# Ensure 'username' is present in all DataFrames
assert 'username' in user_agg.columns, "'username' missing in user_agg"
assert 'username' in genre_dummies.columns, "'username' missing in genre_dummies"
assert 'username' in user_keys.columns, "'username' missing in user_keys"

# Merge DataFrames on 'username'
user_data = pd.merge(user_agg, genre_dummies, on='username', how='left')
user_data = pd.merge(user_data, user_keys, on='username', how='left')


In [21]:
# Load the personality traits data
user_personality = pd.read_csv('data/results.csv')

# Ensure 'username' is in the columns
if 'username' not in user_personality.columns:
    user_personality.rename(columns={'name': 'username'}, inplace=True)

user_personality['username'] = user_personality['username'].astype(str)

# Merge user data with personality traits
user_data = pd.merge(user_data, user_personality, on='username', how='inner')


In [22]:
# Define target columns
target_columns = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']

# Ensure these columns exist in user_data
missing_traits = [col for col in target_columns if col not in user_data.columns]
if missing_traits:
    print(f"Warning: The following personality trait columns are missing: {missing_traits}")

# Features are all columns except 'username' and target columns
feature_columns = [col for col in user_data.columns if col not in ['username'] + target_columns]

# Prepare feature matrix X and target matrix y
X = user_data[feature_columns]
y = user_data[target_columns]

# Handle any remaining missing values
X.fillna(0, inplace=True)


/var/folders/64/s8x7jmb54wq7ldv0c_5hb_wm0000gn/T/ipykernel_64053/3877800092.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace=True)


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Iterate over all personality traits
for trait in target_columns:
    print(f"\nTraining model to predict '{trait}'")
    y_trait = y[trait]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_trait, test_size=0.2, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    
    y_pred = rf_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Mean Squared Error for '{trait}': {mse}")
    print(f"R² Score for '{trait}': {r2}")




Training model to predict 'Openness'
Mean Squared Error for 'Openness': 2731.1076
R² Score for 'Openness': nan

Training model to predict 'Conscientiousness'
Mean Squared Error for 'Conscientiousness': 1270.9225
R² Score for 'Conscientiousness': nan

Training model to predict 'Extraversion'
Mean Squared Error for 'Extraversion': 1.0201000000000104
R² Score for 'Extraversion': nan

Training model to predict 'Agreeableness'


/Users/connorbrezenski/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/connorbrezenski/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/connorbrezenski/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/connorbrezenski/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Mean Squared Error for 'Agreeableness': 2500.0
R² Score for 'Agreeableness': nan

Training model to predict 'Neuroticism'
Mean Squared Error for 'Neuroticism': 0.0016000000000005003
R² Score for 'Neuroticism': nan


/Users/connorbrezenski/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
